In [ ]:
pip install featuretools

In [10]:
import featuretools as ft
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
train = pd.read_csv('train.csv', sep = '|')
test = pd.read_csv('test.csv', delimiter="|")
train.head()




,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [39]:
# saving identifiers
y = train['fraud']
X = train.drop(columns=['fraud']).astype(float)
fit_minmax = MinMaxScaler()
X_encode = pd.get_dummies(X, columns=['trustLevel'], prefix='trustLevel')
X_train_norm_enc = pd.DataFrame(fit_minmax.fit_transform(X_encode), columns=X_encode.columns, index=X_encode.index)
print(X_train_norm_enc.shape)
X_test_encode = pd.get_dummies(test, columns=['trustLevel'], prefix='trustLevel')
X_test_norm_enc = pd.DataFrame(fit_minmax.transform(X_test_encode), columns=X_test_encode.columns, index=X_test_encode.index)

X_manual = X.assign(no_item = X.totalScanTimeInSeconds* X.scannedLineItemsPerSecond)\
                     .drop(columns=['valuePerSecond', 'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition'])
fit_minmax = MinMaxScaler()
X_manual_encode = pd.get_dummies(X_manual, columns=['trustLevel'], prefix='trustLevel')
X_train_manual = pd.DataFrame(fit_minmax.fit_transform(X_manual_encode), columns=X_manual_encode.columns, index=X_manual_encode.index)
print(X_train_manual.shape)

X_test = test.assign(no_item = test.totalScanTimeInSeconds* test.scannedLineItemsPerSecond)\
                     .drop(columns=['valuePerSecond', 'scannedLineItemsPerSecond', 'lineItemVoidsPerPosition'])
X_test_encode = pd.get_dummies(X_test, columns=['trustLevel'], prefix='trustLevel')
X_test_manual = pd.DataFrame(fit_minmax.transform(X_test_encode), columns=X_test_encode.columns, index=X_test_encode.index)
print(X_test_manual.shape)


/Users/sarahfathallah/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(1879, 14)


/Users/sarahfathallah/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(1879, 12)
(498121, 12)


In [40]:
combi = X_train_manual.append(X_test_manual, ignore_index=True, sort=True)

In [103]:
combi.isnull().sum()
combi.head()

,id,grandTotal,lineItemVoids,quantityModifications,scansWithoutRegistration,totalScanTimeInSeconds,trustLevel_1.0,trustLevel_2.0,trustLevel_3.0,trustLevel_4.0,trustLevel_5.0,trustLevel_6.0
0,0,0.547174,0.636364,0.6,0.0,0.575178,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0.273637,0.454545,0.8,0.2,0.057955,0.0,0.0,1.0,0.0,0.0,0.0
2,2,0.621811,0.272727,1.0,1.0,0.827775,0.0,0.0,1.0,0.0,0.0,0.0
3,3,0.923462,0.727273,0.8,0.4,0.978130,0.0,0.0,0.0,0.0,0.0,1.0
4,4,0.815608,0.272727,0.4,0.7,0.234008,0.0,0.0,0.0,0.0,1.0,0.0


In [98]:
combi['id'] = combi['id']
combi.drop(['id'], axis=1, inplace=True)
combi.drop(['trustLevel_1'], axis=1, inplace=True)
combi.drop(['trustLevel_2'], axis=1, inplace=True)
combi.drop(['trustLevel_3'], axis=1, inplace=True)
combi.drop(['trustLevel_4'], axis=1, inplace=True)
combi.drop(['trustLevel_5'], axis=1, inplace=True)
combi.drop(['trustLevel_6'], axis=1, inplace=True)

In [99]:
# creating and entity set 'es'
es = ft.EntitySet(id = 'dmc')

es.entity_from_dataframe(entity_id='dmc',dataframe = combi, index = 'id')
es.normalize_entity(base_entity_id='dmc', new_entity_id='outlet', index = 'id',
                   additional_variables = ['lineItemVoids', 'quantityModifications', 'scansWithoutRegistration', 'totalScanTimeInSeconds'])
print(es)


2019-05-05 22:39:32,542 featuretools.entityset - WARNING    index id not found in dataframe, creating new integer column
Entityset: dmc
  Entities:
    dmc [Rows: 500000, Columns: 8]
    outlet [Rows: 500000, Columns: 5]
  Relationships:
    dmc.id -> outlet.id


In [100]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'dmc', 
max_depth = 2, 
verbose = 1, 
n_jobs = 3)


Built 53 features
EntitySet scattered to workers in 7.227 seconds
Elapsed: 18:04 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks


In [102]:
feature_matrix.columns


Index(['grandTotal', 'trustLevel_1.0', 'trustLevel_2.0', 'trustLevel_3.0',
       'trustLevel_4.0', 'trustLevel_5.0', 'trustLevel_6.0',
       'outlet.lineItemVoids', 'outlet.quantityModifications',
       'outlet.scansWithoutRegistration', 'outlet.totalScanTimeInSeconds',
       'outlet.SUM(dmc.grandTotal)', 'outlet.SUM(dmc.trustLevel_1.0)',
       'outlet.SUM(dmc.trustLevel_2.0)', 'outlet.SUM(dmc.trustLevel_3.0)',
       'outlet.SUM(dmc.trustLevel_4.0)', 'outlet.SUM(dmc.trustLevel_5.0)',
       'outlet.SUM(dmc.trustLevel_6.0)', 'outlet.STD(dmc.grandTotal)',
       'outlet.STD(dmc.trustLevel_1.0)', 'outlet.STD(dmc.trustLevel_2.0)',
       'outlet.STD(dmc.trustLevel_3.0)', 'outlet.STD(dmc.trustLevel_4.0)',
       'outlet.STD(dmc.trustLevel_5.0)', 'outlet.STD(dmc.trustLevel_6.0)',
       'outlet.MAX(dmc.grandTotal)', 'outlet.MAX(dmc.trustLevel_1.0)',
       'outlet.MAX(dmc.trustLevel_2.0)', 'outlet.MAX(dmc.trustLevel_3.0)',
       'outlet.MAX(dmc.trustLevel_4.0)', 'outlet.MAX(dmc.trustL

In [104]:
feature_matrix.head()

,grandTotal,trustLevel_1.0,trustLevel_2.0,trustLevel_3.0,trustLevel_4.0,trustLevel_5.0,trustLevel_6.0,outlet.lineItemVoids,outlet.quantityModifications,outlet.scansWithoutRegistration,...,outlet.MIN(dmc.trustLevel_4.0),outlet.MIN(dmc.trustLevel_5.0),outlet.MIN(dmc.trustLevel_6.0),outlet.MEAN(dmc.grandTotal),outlet.MEAN(dmc.trustLevel_1.0),outlet.MEAN(dmc.trustLevel_2.0),outlet.MEAN(dmc.trustLevel_3.0),outlet.MEAN(dmc.trustLevel_4.0),outlet.MEAN(dmc.trustLevel_5.0),outlet.MEAN(dmc.trustLevel_6.0)
id,,,,,,,,,,,,,,,,,,,,,
0,0.547174,0.0,0.0,0.0,0.0,1.0,0.0,0.636364,0.6,0.0,...,0.0,1.0,0.0,0.547174,0.0,0.0,0.0,0.0,1.0,0.0
1,0.273637,0.0,0.0,1.0,0.0,0.0,0.0,0.454545,0.8,0.2,...,0.0,0.0,0.0,0.273637,0.0,0.0,1.0,0.0,0.0,0.0
2,0.621811,0.0,0.0,1.0,0.0,0.0,0.0,0.272727,1.0,1.0,...,0.0,0.0,0.0,0.621811,0.0,0.0,1.0,0.0,0.0,0.0
3,0.923462,0.0,0.0,0.0,0.0,0.0,1.0,0.727273,0.8,0.4,...,0.0,0.0,1.0,0.923462,0.0,0.0,0.0,0.0,0.0,1.0
4,0.815608,0.0,0.0,0.0,0.0,1.0,0.0,0.272727,0.4,0.7,...,0.0,1.0,0.0,0.815608,0.0,0.0,0.0,0.0,1.0,0.0


In [128]:
feature_matrix = feature_matrix.reindex(index=combi['id'])
feature_matrix = feature_matrix.reset_index()
feature_matrix = feature_matrix.replace(np.nan, 0)


In [129]:
from catboost import CatBoostRegressor

In [77]:
pip install catboost

    100% |████████████████████████████████| 8.0MB 1.7MB/s 
Note: you may need to restart the kernel to use updated packages.


In [130]:
categorical_features = np.where(feature_matrix.dtypes == 'object')[0]

for i in categorical_features:
    feature_matrix.iloc[:,i] = feature_matrix.iloc[:,i].astype('str')

In [131]:
feature_matrix.drop(['id'], axis=1, inplace=True)
train = feature_matrix[:8523]
test = feature_matrix[8523:]


In [118]:
# removing uneccesary variables
train.drop(['grandTotal'], axis=1, inplace=True)
test.drop(['grandTotal'], axis=1, inplace=True)

In [132]:
categorical_features = np.where(train.dtypes == 'object')[0]
feature_matrix.head()

,grandTotal,trustLevel_1.0,trustLevel_2.0,trustLevel_3.0,trustLevel_4.0,trustLevel_5.0,trustLevel_6.0,outlet.lineItemVoids,outlet.quantityModifications,outlet.scansWithoutRegistration,...,outlet.MIN(dmc.trustLevel_4.0),outlet.MIN(dmc.trustLevel_5.0),outlet.MIN(dmc.trustLevel_6.0),outlet.MEAN(dmc.grandTotal),outlet.MEAN(dmc.trustLevel_1.0),outlet.MEAN(dmc.trustLevel_2.0),outlet.MEAN(dmc.trustLevel_3.0),outlet.MEAN(dmc.trustLevel_4.0),outlet.MEAN(dmc.trustLevel_5.0),outlet.MEAN(dmc.trustLevel_6.0)
0,0.547174,0.0,0.0,0.0,0.0,1.0,0.0,0.636364,0.6,0.0,...,0.0,1.0,0.0,0.547174,0.0,0.0,0.0,0.0,1.0,0.0
1,0.273637,0.0,0.0,1.0,0.0,0.0,0.0,0.454545,0.8,0.2,...,0.0,0.0,0.0,0.273637,0.0,0.0,1.0,0.0,0.0,0.0
2,0.621811,0.0,0.0,1.0,0.0,0.0,0.0,0.272727,1.0,1.0,...,0.0,0.0,0.0,0.621811,0.0,0.0,1.0,0.0,0.0,0.0
3,0.923462,0.0,0.0,0.0,0.0,0.0,1.0,0.727273,0.8,0.4,...,0.0,0.0,1.0,0.923462,0.0,0.0,0.0,0.0,0.0,1.0
4,0.815608,0.0,0.0,0.0,0.0,1.0,0.0,0.272727,0.4,0.7,...,0.0,1.0,0.0,0.815608,0.0,0.0,0.0,0.0,1.0,0.0


In [134]:
from sklearn.model_selection import train_test_split

# splitting train data into training and validation set
xtrain, xvalid, ytrain, yvalid = train_test_split(train, train, test_size=0.25, random_state=11)
xtrain.head()

,grandTotal,trustLevel_1.0,trustLevel_2.0,trustLevel_3.0,trustLevel_4.0,trustLevel_5.0,trustLevel_6.0,outlet.lineItemVoids,outlet.quantityModifications,outlet.scansWithoutRegistration,...,outlet.MIN(dmc.trustLevel_4.0),outlet.MIN(dmc.trustLevel_5.0),outlet.MIN(dmc.trustLevel_6.0),outlet.MEAN(dmc.grandTotal),outlet.MEAN(dmc.trustLevel_1.0),outlet.MEAN(dmc.trustLevel_2.0),outlet.MEAN(dmc.trustLevel_3.0),outlet.MEAN(dmc.trustLevel_4.0),outlet.MEAN(dmc.trustLevel_5.0),outlet.MEAN(dmc.trustLevel_6.0)
7438,0.792796,0.0,0.0,0.0,0.0,0.0,0.0,0.272727,0.8,0.9,...,0.0,0.0,0.0,0.792796,0.0,0.0,0.0,0.0,0.0,0.0
7470,0.734067,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.2,0.1,...,0.0,0.0,0.0,0.734067,0.0,0.0,0.0,0.0,0.0,0.0
1825,0.350275,1.0,0.0,0.0,0.0,0.0,0.0,0.727273,0.2,1.0,...,0.0,0.0,0.0,0.350275,1.0,0.0,0.0,0.0,0.0,0.0
3072,0.674737,0.0,0.0,0.0,0.0,0.0,0.0,0.454545,0.2,0.1,...,0.0,0.0,0.0,0.674737,0.0,0.0,0.0,0.0,0.0,0.0
446,0.085943,1.0,0.0,0.0,0.0,0.0,0.0,0.909091,0.4,0.9,...,0.0,0.0,0.0,0.085943,1.0,0.0,0.0,0.0,0.0,0.0


In [135]:
model_cat = CatBoostRegressor(iterations=100, learning_rate=0.3, depth=6, eval_metric='RMSE', random_seed=7)

# training model
model_cat.fit(xtrain, ytrain, cat_features=categorical_features, use_best_model=True)

You should provide test set for use best model. use_best_model parameter swiched to false value.


0:	learn: 0.4105416	total: 58.4ms	remaining: 5.78s
1:	learn: 0.2898470	total: 94.1ms	remaining: 4.61s
2:	learn: 0.2055601	total: 136ms	remaining: 4.39s
3:	learn: 0.1464866	total: 179ms	remaining: 4.3s
4:	learn: 0.1047866	total: 236ms	remaining: 4.48s
5:	learn: 0.0755291	total: 297ms	remaining: 4.66s
6:	learn: 0.0552803	total: 398ms	remaining: 5.29s
7:	learn: 0.0411850	total: 656ms	remaining: 7.54s
8:	learn: 0.0316636	total: 824ms	remaining: 8.33s
9:	learn: 0.0255039	total: 917ms	remaining: 8.25s
10:	learn: 0.0208058	total: 982ms	remaining: 7.95s
11:	learn: 0.0184612	total: 1.04s	remaining: 7.61s
12:	learn: 0.0162290	total: 1.13s	remaining: 7.54s
13:	learn: 0.0149347	total: 1.16s	remaining: 7.12s
14:	learn: 0.0139521	total: 1.19s	remaining: 6.75s
15:	learn: 0.0130376	total: 1.23s	remaining: 6.46s
16:	learn: 0.0127072	total: 1.29s	remaining: 6.28s
17:	learn: 0.0125193	total: 1.78s	remaining: 8.11s
18:	learn: 0.0122603	total: 1.83s	remaining: 7.8s
19:	learn: 0.0118642	total: 2.13s	remaini

In [136]:
model_cat.score(xvalid, yvalid)

0.5346777007978177